In [4]:
from pathlib import Path
import re
import pandas as pd
import os
import numpy as np

In [5]:
data_root = r"C:/Users/hayashi/doutorado/mac6958--dados-redes/webrtc"

In [6]:
Path(data_root)

WindowsPath('C:/Users/hayashi/doutorado/mac6958--dados-redes/webrtc')

In [7]:
print(Path.cwd())

C:\Users\hayashi\doutorado\mac6958--dados-redes\webrtc\notebooks


In [36]:
files = [f for f in Path(data_root, 'data/raw/Facial_expression_features_dataset').rglob('TC*.csv')]
files[:10]

[WindowsPath('C:/Users/hayashi/doutorado/mac6958--dados-redes/webrtc/data/raw/Facial_expression_features_dataset/User1/TC10_1000-500-15.csv'),
 WindowsPath('C:/Users/hayashi/doutorado/mac6958--dados-redes/webrtc/data/raw/Facial_expression_features_dataset/User1/TC11_0-0-30.csv'),
 WindowsPath('C:/Users/hayashi/doutorado/mac6958--dados-redes/webrtc/data/raw/Facial_expression_features_dataset/User1/TC12_500-0-30.csv'),
 WindowsPath('C:/Users/hayashi/doutorado/mac6958--dados-redes/webrtc/data/raw/Facial_expression_features_dataset/User1/TC13_1000-0-30.csv'),
 WindowsPath('C:/Users/hayashi/doutorado/mac6958--dados-redes/webrtc/data/raw/Facial_expression_features_dataset/User1/TC14_500-500-30.csv'),
 WindowsPath('C:/Users/hayashi/doutorado/mac6958--dados-redes/webrtc/data/raw/Facial_expression_features_dataset/User1/TC15_1000-500-30.csv'),
 WindowsPath('C:/Users/hayashi/doutorado/mac6958--dados-redes/webrtc/data/raw/Facial_expression_features_dataset/User1/TC1_0-0-0.csv'),
 WindowsPath('C:/

In [9]:
for f in files:
    print(str(f).split('\\')[-2:], f.name.split('[_-]'), f.name )

['User1', 'TC10_1000-500-15.csv'] ['TC10_1000-500-15.csv'] TC10_1000-500-15.csv
['User1', 'TC11_0-0-30.csv'] ['TC11_0-0-30.csv'] TC11_0-0-30.csv
['User1', 'TC12_500-0-30.csv'] ['TC12_500-0-30.csv'] TC12_500-0-30.csv
['User1', 'TC13_1000-0-30.csv'] ['TC13_1000-0-30.csv'] TC13_1000-0-30.csv
['User1', 'TC14_500-500-30.csv'] ['TC14_500-500-30.csv'] TC14_500-500-30.csv
['User1', 'TC15_1000-500-30.csv'] ['TC15_1000-500-30.csv'] TC15_1000-500-30.csv
['User1', 'TC1_0-0-0.csv'] ['TC1_0-0-0.csv'] TC1_0-0-0.csv
['User1', 'TC2_500-0-0.csv'] ['TC2_500-0-0.csv'] TC2_500-0-0.csv
['User1', 'TC3_1000-0-0.csv'] ['TC3_1000-0-0.csv'] TC3_1000-0-0.csv
['User1', 'TC4_500-500-0.csv'] ['TC4_500-500-0.csv'] TC4_500-500-0.csv
['User1', 'TC5_1000-500-0.csv'] ['TC5_1000-500-0.csv'] TC5_1000-500-0.csv
['User1', 'TC6_0-0-15.csv'] ['TC6_0-0-15.csv'] TC6_0-0-15.csv
['User1', 'TC7_500-0-15.csv'] ['TC7_500-0-15.csv'] TC7_500-0-15.csv
['User1', 'TC8_1000-0-15.csv'] ['TC8_1000-0-15.csv'] TC8_1000-0-15.csv
['User1', 'TC9_

In [10]:
re.split('[_\-.]', 'TC10_1000-500-15.csv')[:-1]

['TC10', '1000', '500', '15']

In [11]:
def label_features(f):
    user = str(f).split('\\')[-2]
    from_name= re.split('[_\-.]', f.name)[:-1]
    return from_name[0:1]+ [user]+ from_name[1:-1]

In [12]:
label_features(files[0])

['TC10', 'User1', '1000', '500']

In [43]:
os.path.basename(files[0])

'TC10_1000-500-15.csv'

In [48]:
def read_and_prepare_data(file_paths):
    data_info_list = []
    
    for file_path in file_paths:
        # Extract information from the file name
        file_namd_and_user = '/'.join(str(file_path).split('\\')[-2:])
        match = re.match(r'User(\d+)/TC(\d+)_(\d+)-(\d+)-(\d+).csv', file_namd_and_user)
        if match:
            user_id = int(match.group(1))
            test_condition = int(match.group(2))
            delay = int(match.group(3))
            jitter = int(match.group(4))
            packet_loss_rate = int(match.group(5))
        else:
            print( 'Oops, not mach. Skip ', file_path)
            continue  # Skip files that do not match the pattern
        
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Select only the feature columns (starting from the 6th column)
        features = df.iloc[:, 5:].values
        
        # Convert to numpy array
        features_array = np.array(features, dtype=float)
        
        # Append the data and the extracted information to the list
        data_info = {
            'user_id': user_id,
            'test_condition': test_condition,
            'delay': delay,
            'jitter': jitter,
            'packet_loss_rate': packet_loss_rate,
            'features': features_array
        }
        data_info_list.append(data_info)
    
    return data_info_list

In [51]:
data_raw = read_and_prepare_data(files)

In [53]:
len(data_raw)

300

In [54]:
data_raw[0]['features'].shape

(3676, 323)

In [55]:
import pickle

In [56]:
def save_data(data, filename):
    with open(filename, 'wb') as file:
        pickle.dump(data, file)

processed_file = Path(data_root, 'data/processed/facial-1001.pkl')
save_data(data_raw, processed_file)

In [24]:
def load_data(filename):
    with open(filename, 'rb') as file:
        return pickle.load(file)

In [25]:
loaded_data = load_data(processed_file)

In [26]:
len(loaded_data)

300

In [27]:
loaded_data[0]['features'].shape

(3676, 323)

In [28]:
[d['delay'] for d in data_raw]

[1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,